In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit.aqua import QuantumInstance
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import pylab
from qiskit.circuit import ParameterVector
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, Molecule
from qiskit.chemistry.transformations import FermionicTransformation, FermionicQubitMappingType
from qiskit.chemistry.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.algorithms import VQE
from qiskit.circuit.library import TwoLocal
import numpy as np

In [2]:
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

backend = Aer.get_backend("qasm_simulator") 
print(backend)

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


qasm_simulator


In [3]:
#Set the seed for reproducible results
aqua_globals.random_seed = 100
seed_simulator = 100
seed_transpiler=100
np.random.seed(100)

In [4]:
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
transformation = FermionicTransformation(qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER)

In [5]:
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

In [6]:
numpy_solver = NumPyMinimumEigensolver()
calc = GroundStateEigensolver(transformation, numpy_solver)
res = calc.solve(driver)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
    - frozen energy part: [0.0  0.0  0.0]
    - particle hole part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.
                 (debye): [0.0  0.0  0.0]  Total: 0.
 


In [7]:
def quantum_state_preparation(parameters, gseq):
    for gorder, elist in gseq:

        circuit = QuantumCircuit(4)
        circuit.ry(parameters[0], 0)
        circuit.ry(parameters[1], 1)
        circuit.ry(parameters[2], 2)
        circuit.ry(parameters[3], 3)
        circuit.barrier()
        for i, j in enumerate(gorder):
            if gorder[i] == 'cx':
                circuit.cx(elist[i][0], elist[i][1])
            else:
                circuit.cz(elist[i][0], elist[i][1])
        circuit.barrier()
        circuit.ry(parameters[4], 0)
        circuit.ry(parameters[5], 1)
        circuit.ry(parameters[6], 2)
        circuit.ry(parameters[7], 3)
        return circuit

In [8]:
params = ParameterVector('θ', 8) 

#CXCZ circuit experiment
gatelist = [(['cx', 'cz', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cx', 'cz', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cx', 'cx', 'cz', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cx', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cx', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cz', 'cx', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)])]

for i in range(len(gatelist)):
    gates = [gatelist[i]]
    circ = quantum_state_preparation(params,gates)
    quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed_simulator, seed_transpiler=seed_transpiler)
    optimizer = SPSA(maxiter=200)
    vqe_solver = VQE(var_form = circ, quantum_instance = quantum_instance, optimizer= optimizer, callback=store_intermediate_result) 
    calc = GroundStateEigensolver(transformation, vqe_solver)
    result = calc.solve(driver)
    print(circ)
    print(result)

     ┌──────────┐ ░                  ░ ┌──────────┐
q_0: ┤ RY(θ[0]) ├─░───────────■───■──░─┤ RY(θ[4]) ├
     ├──────────┤ ░         ┌─┴─┐ │  ░ ├──────────┤
q_1: ┤ RY(θ[1]) ├─░───────■─┤ X ├─┼──░─┤ RY(θ[5]) ├
     ├──────────┤ ░       │ └───┘ │  ░ ├──────────┤
q_2: ┤ RY(θ[2]) ├─░───■───■───────┼──░─┤ RY(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐         │  ░ ├──────────┤
q_3: ┤ RY(θ[3]) ├─░─┤ X ├─────────■──░─┤ RY(θ[7]) ├
     └──────────┘ ░ └───┘            ░ └──────────┘
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.840541721431
  - computed part:      -1.840541721431
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.120572726982
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 1.999 S: 0.005 S^2: 0.005 M: 0.001
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.39166696]
  